In [4]:
from matplotlib.patches import Rectangle
from mtcnn.mtcnn import MTCNN

# Directory paths
train_directory = '/kaggle/working/train'

# Get a list of all folders in the train directory
all_folders = [folder for folder in os.listdir(train_directory) if os.path.isdir(os.path.join(train_directory, folder))]

# Randomly select two folders
selected_folders = random.sample(all_folders, 2)

# Initialize MTCNN detector
detector = MTCNN()

# Iterate through selected folders
for folder in selected_folders:
    folder_path = os.path.join(train_directory, folder)
    
    # Get a list of all images in the folder
    all_images = [img for img in os.listdir(folder_path) if img.endswith('.jpg')]
    
    # Randomly select one image
    selected_image = random.choice(all_images)
    image_path = os.path.join(folder_path, selected_image)
    
    # Read the image
    image = cv2.imread(image_path)
    image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    # Detect faces using MTCNN
    faces = detector.detect_faces(image)
    
    # Display image with bounding boxes around detected faces
    plt.figure(figsize=(8, 3))
    
    # Display original image
    plt.subplot(1, 2, 1)
    plt.imshow(image_rgb)
    plt.title('Original Image')
    
    # Display image with bounding boxes
    plt.subplot(1, 2, 2)
    plt.imshow(image_rgb)
    
    for face in faces:
        x, y, width, height = face['box']
        rect = Rectangle((x, y), width, height, fill=False, color='red')
        plt.gca().add_patch(rect)
        
        # Display additional keypoints
        for key, value in face['keypoints'].items():
            plt.scatter(value[0], value[1], s=30, color='blue', marker='o')
            plt.text(value[0] + 5, value[1], key, color='blue')
    
    plt.title('Detected Faces with Keypoints')
    plt.show()

    # Display metadata of detected faces
    print(f"Metadata of detected faces in {folder}/{selected_image}:")
    for i, face in enumerate(faces):
        print(f"Face {i + 1}:")
        print(f"   Confidence: {face['confidence']:.2f}")
        print(f"   Bounding Box: {face['box']}")
        print(f"   Keypoints: {face['keypoints']}")
        print()

ModuleNotFoundError: No module named 'tensorflow'

In [5]:
from flask import Flask, render_template, request, jsonify, send_from_directory
import os
import cv2
import numpy as np
from mtcnn import MTCNN
import base64
from io import BytesIO
from PIL import Image, ImageDraw, ImageFont
import json

app = Flask(__name__)
app.config['UPLOAD_FOLDER'] = 'uploads'
app.config['MAX_CONTENT_LENGTH'] = 16 * 1024 * 1024  # 16MB max file size

# Create upload directory if it doesn't exist
os.makedirs(app.config['UPLOAD_FOLDER'], exist_ok=True)

# Initialize MTCNN detector
detector = MTCNN(device="CPU:0")

# Face Detection Ontology
FACE_ONTOLOGY = {
    "Face": {
        "description": "A human facial region detected in an image",
        "properties": {
            "bounding_box": {
                "type": "Rectangle",
                "description": "Rectangular boundary containing the face",
                "coordinates": ["x", "y", "width", "height"]
            },
            "confidence": {
                "type": "Float",
                "description": "Detection confidence score (0.0 to 1.0)",
                "range": [0.0, 1.0]
            },
            "keypoints": {
                "type": "FacialKeypoints",
                "description": "Anatomical landmarks on the face"
            }
        }
    },
    "FacialKeypoints": {
        "description": "Key anatomical points on a human face",
        "landmarks": {
            "left_eye": {
                "description": "Center of the left eye (from viewer's perspective)",
                "anatomical_region": "Ocular",
                "coordinates": ["x", "y"]
            },
            "right_eye": {
                "description": "Center of the right eye (from viewer's perspective)",
                "anatomical_region": "Ocular",
                "coordinates": ["x", "y"]
            },
            "nose": {
                "description": "Tip of the nose",
                "anatomical_region": "Nasal",
                "coordinates": ["x", "y"]
            },
            "mouth_left": {
                "description": "Left corner of the mouth",
                "anatomical_region": "Oral",
                "coordinates": ["x", "y"]
            },
            "mouth_right": {
                "description": "Right corner of the mouth",
                "anatomical_region": "Oral",
                "coordinates": ["x", "y"]
            }
        }
    }
}


def allowed_file(filename):
    """Check if file extension is allowed"""
    ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg', 'gif', 'bmp'}
    return '.' in filename and \
        filename.rsplit('.', 1)[1].lower() in ALLOWED_EXTENSIONS


def draw_detections(image, detections):
    """Draw bounding boxes and keypoints on the image"""
    draw = ImageDraw.Draw(image)

    # Define colors
    box_color = (0, 255, 0)  # Green for bounding box
    keypoint_colors = {
        'left_eye': (255, 0, 0),  # Red
        'right_eye': (255, 0, 0),  # Red
        'nose': (0, 0, 255),  # Blue
        'mouth_left': (255, 255, 0),  # Yellow
        'mouth_right': (255, 255, 0)  # Yellow
    }

    for detection in detections:
        # Draw bounding box
        box = detection['box']
        x, y, w, h = box
        draw.rectangle([x, y, x + w, y + h], outline=box_color, width=3)

        # Draw confidence score
        confidence_text = f"Confidence: {detection['confidence']:.3f}"
        draw.text((x, y - 20), confidence_text, fill=box_color)

        # Draw keypoints
        keypoints = detection['keypoints']
        for keypoint_name, (kx, ky) in keypoints.items():
            color = keypoint_colors.get(keypoint_name, (255, 255, 255))
            # Draw circle for keypoint
            radius = 4
            draw.ellipse([kx - radius, ky - radius, kx + radius, ky + radius],
                         fill=color, outline=(0, 0, 0), width=1)
            # Draw label
            draw.text((kx + 5, ky - 10), keypoint_name, fill=color)

    return image


def annotate_with_ontology(detections):
    """Add ontological information to detections"""
    annotated_results = []

    for i, detection in enumerate(detections):
        annotated_detection = {
            "face_id": i + 1,
            "ontology_class": "Face",
            "detection_data": detection,
            "semantic_annotation": {
                "anatomical_regions": {
                    "ocular": {
                        "left_eye": detection['keypoints']['left_eye'],
                        "right_eye": detection['keypoints']['right_eye']
                    },
                    "nasal": {
                        "nose": detection['keypoints']['nose']
                    },
                    "oral": {
                        "mouth_left": detection['keypoints']['mouth_left'],
                        "mouth_right": detection['keypoints']['mouth_right']
                    }
                },
                "quality_metrics": {
                    "detection_confidence": detection['confidence'],
                    "quality_assessment": "high" if detection['confidence'] > 0.95 else "medium" if detection[
                                                                                                        'confidence'] > 0.8 else "low"
                }
            }
        }
        annotated_results.append(annotated_detection)

    return annotated_results


@app.route('/')
def index():
    return render_template('index.html')


@app.route('/ontology')
def get_ontology():
    """Return the face detection ontology"""
    return jsonify(FACE_ONTOLOGY)


@app.route('/upload', methods=['POST'])
def upload_file():
    if 'file' not in request.files:
        return jsonify({'error': 'No file selected'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No file selected'}), 400

    if file and allowed_file(file.filename):
        try:
            # Read image
            image_bytes = file.read()
            image = Image.open(BytesIO(image_bytes))

            # Convert to RGB if necessary
            if image.mode != 'RGB':
                image = image.convert('RGB')

            # Convert PIL image to numpy array for MTCNN
            image_array = np.array(image)

            # Detect faces
            detections = detector.detect_faces(image_array)

            if not detections:
                return jsonify({
                    'message': 'No faces detected in the image',
                    'detections': [],
                    'annotated_results': []
                })

            # Draw detections on image
            annotated_image = draw_detections(image.copy(), detections)

            # Convert annotated image to base64
            buffered = BytesIO()
            annotated_image.save(buffered, format="JPEG")
            img_str = base64.b64encode(buffered.getvalue()).decode()

            # Add ontological annotations
            annotated_results = annotate_with_ontology(detections)

            return jsonify({
                'success': True,
                'message': f'Detected {len(detections)} face(s)',
                'detections': detections,
                'annotated_results': annotated_results,
                'annotated_image': f'data:image/jpeg;base64,{img_str}'
            })

        except Exception as e:
            return jsonify({'error': f'Error processing image: {str(e)}'}), 500

    return jsonify({'error': 'Invalid file type'}), 400


if __name__ == '__main__':
    # Create templates directory and index.html if they don't exist
    os.makedirs('templates', exist_ok=True)

    html_content = '''
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>MTCNN Face Detection with Ontology</title>
    <style>
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            max-width: 1200px;
            margin: 0 auto;
            padding: 20px;
            background-color: #f5f5f5;
        }
        .container {
            background: white;
            padding: 30px;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
        }
        h1 {
            color: #333;
            text-align: center;
            margin-bottom: 30px;
        }
        .upload-section {
            border: 2px dashed #ddd;
            border-radius: 10px;
            padding: 40px;
            text-align: center;
            margin-bottom: 30px;
            transition: border-color 0.3s;
        }
        .upload-section:hover {
            border-color: #007bff;
        }
        .file-input {
            margin: 20px 0;
        }
        .btn {
            background-color: #007bff;
            color: white;
            padding: 12px 24px;
            border: none;
            border-radius: 5px;
            cursor: pointer;
            font-size: 16px;
            transition: background-color 0.3s;
        }
        .btn:hover {
            background-color: #0056b3;
        }
        .btn:disabled {
            background-color: #6c757d;
            cursor: not-allowed;
        }
        .results {
            margin-top: 30px;
        }
        .result-image {
            max-width: 100%;
            height: auto;
            border-radius: 10px;
            box-shadow: 0 2px 10px rgba(0,0,0,0.1);
            margin: 20px 0;
        }
        .detection-info {
            background: #f8f9fa;
            padding: 20px;
            border-radius: 10px;
            margin: 20px 0;
        }
        .face-card {
            background: white;
            border: 1px solid #ddd;
            border-radius: 8px;
            padding: 15px;
            margin: 10px 0;
        }
        .keypoint {
            display: inline-block;
            background: #e9ecef;
            padding: 5px 10px;
            border-radius: 15px;
            margin: 3px;
            font-size: 12px;
        }
        .confidence {
            font-weight: bold;
            color: #28a745;
        }
        .ontology-section {
            background: #fff3cd;
            border: 1px solid #ffeaa7;
            border-radius: 10px;
            padding: 20px;
            margin: 20px 0;
        }
        .loading {
            display: none;
            text-align: center;
            margin: 20px 0;
        }
        .spinner {
            border: 4px solid #f3f3f3;
            border-top: 4px solid #007bff;
            border-radius: 50%;
            width: 40px;
            height: 40px;
            animation: spin 1s linear infinite;
            margin: 0 auto;
        }
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
        .error {
            color: #dc3545;
            background: #f8d7da;
            border: 1px solid #f5c6cb;
            padding: 15px;
            border-radius: 5px;
            margin: 20px 0;
        }
        .success {
            color: #155724;
            background: #d4edda;
            border: 1px solid #c3e6cb;
            padding: 15px;
            border-radius: 5px;
            margin: 20px 0;
        }
    </style>
</head>
<body>
    <div class="container">
        <h1>🔍 MTCNN Face Detection with Ontology</h1>

        <div class="upload-section">
            <h3>Upload an Image for Face Detection</h3>
            <p>Supported formats: JPG, JPEG, PNG, GIF, BMP</p>
            <input type="file" id="imageFile" class="file-input" accept="image/*">
            <br>
            <button onclick="uploadImage()" class="btn" id="uploadBtn">
                Detect Faces
            </button>
        </div>

        <div class="loading" id="loading">
            <div class="spinner"></div>
            <p>Processing image...</p>
        </div>

        <div id="results" class="results"></div>
    </div>

    <script>
        function uploadImage() {
            const fileInput = document.getElementById('imageFile');
            const file = fileInput.files[0];

            if (!file) {
                alert('Please select an image file first.');
                return;
            }

            const formData = new FormData();
            formData.append('file', file);

            // Show loading
            document.getElementById('loading').style.display = 'block';
            document.getElementById('uploadBtn').disabled = true;
            document.getElementById('results').innerHTML = '';

            fetch('/upload', {
                method: 'POST',
                body: formData
            })
            .then(response => response.json())
            .then(data => {
                // Hide loading
                document.getElementById('loading').style.display = 'none';
                document.getElementById('uploadBtn').disabled = false;

                if (data.error) {
                    document.getElementById('results').innerHTML = 
                        `<div class="error">Error: ${data.error}</div>`;
                    return;
                }

                displayResults(data);
            })
            .catch(error => {
                console.error('Error:', error);
                document.getElementById('loading').style.display = 'none';
                document.getElementById('uploadBtn').disabled = false;
                document.getElementById('results').innerHTML = 
                    `<div class="error">Error uploading image: ${error.message}</div>`;
            });
        }

        function displayResults(data) {
            const resultsDiv = document.getElementById('results');

            if (data.detections.length === 0) {
                resultsDiv.innerHTML = `
                    <div class="success">${data.message}</div>
                `;
                return;
            }

            let html = `
                <div class="success">${data.message}</div>

                <h3>📸 Annotated Image</h3>
                <img src="${data.annotated_image}" alt="Annotated Image" class="result-image">

                <h3>🎯 Detection Results</h3>
            `;

            data.annotated_results.forEach((result, index) => {
                const detection = result.detection_data;
                const semantic = result.semantic_annotation;

                html += `
                    <div class="face-card">
                        <h4>Face ${result.face_id}</h4>
                        <p><strong>Confidence:</strong> <span class="confidence">${(detection.confidence * 100).toFixed(1)}%</span></p>
                        <p><strong>Quality:</strong> ${semantic.quality_metrics.quality_assessment}</p>
                        <p><strong>Bounding Box:</strong> [${detection.box.join(', ')}]</p>

                        <h5>🔍 Facial Keypoints:</h5>
                        <div>
                `;

                Object.entries(detection.keypoints).forEach(([name, coords]) => {
                    html += `<span class="keypoint">${name.replace('_', ' ')}: (${coords[0]}, ${coords[1]})</span>`;
                });

                html += `
                        </div>

                        <h5>🧠 Ontological Classification:</h5>
                        <p><strong>Class:</strong> ${result.ontology_class}</p>
                        <p><strong>Anatomical Regions:</strong></p>
                        <ul>
                            <li><strong>Ocular:</strong> Left eye (${semantic.anatomical_regions.ocular.left_eye}), Right eye (${semantic.anatomical_regions.ocular.right_eye})</li>
                            <li><strong>Nasal:</strong> Nose (${semantic.anatomical_regions.nasal.nose})</li>
                            <li><strong>Oral:</strong> Mouth left (${semantic.anatomical_regions.oral.mouth_left}), Mouth right (${semantic.anatomical_regions.oral.mouth_right})</li>
                        </ul>
                    </div>
                `;
            });

            html += `
                <div class="ontology-section">
                    <h3>📚 Ontology Information</h3>
                    <p>This application uses a structured ontology to classify and annotate facial features:</p>
                    <ul>
                        <li><strong>Face:</strong> The main detected object with bounding box, confidence, and keypoints</li>
                        <li><strong>Anatomical Regions:</strong> Ocular (eyes), Nasal (nose), and Oral (mouth) regions</li>
                        <li><strong>Quality Metrics:</strong> Assessment based on detection confidence</li>
                    </ul>
                    <p><small>View the complete ontology structure at: <a href="/ontology" target="_blank">/ontology</a></small></p>
                </div>
            `;

            resultsDiv.innerHTML = html;
        }

        // Allow drag and drop
        const uploadSection = document.querySelector('.upload-section');

        uploadSection.addEventListener('dragover', (e) => {
            e.preventDefault();
            uploadSection.style.borderColor = '#007bff';
        });

        uploadSection.addEventListener('dragleave', (e) => {
            e.preventDefault();
            uploadSection.style.borderColor = '#ddd';
        });

        uploadSection.addEventListener('drop', (e) => {
            e.preventDefault();
            uploadSection.style.borderColor = '#ddd';

            const files = e.dataTransfer.files;
            if (files.length > 0) {
                document.getElementById('imageFile').files = files;
            }
        });
    </script>
</body>
</html>
    '''

    with open('templates/index.html', 'w') as f:
        f.write(html_content)

    print("Starting Flask app...")
    print("Make sure to install required packages:")
    print("pip install flask mtcnn pillow opencv-python")
    print("\nAccess the application at: http://localhost:5000")

    app.run(debug=True)

ModuleNotFoundError: No module named 'cv2'